In [1]:
import torch
from torchvision import datasets, transforms
from torch import nn, optim
import syft as sy
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Subset
import numpy as np
#import tensorflow as tf

import torchvision
import torchvision.transforms as transforms

C:\Users\Raffaele\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
transform_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset_M = MNIST(root='./data', train=True, download=True, transform=transform_mnist)
test_dataset_M = MNIST(root='./data', train=False, download=True, transform=transform_mnist)


# Extract features (images) and labels
#x_train = train_dataset.data.float()  # Convert images to float tensor
#y_train = train_dataset.targets       # Extract labels

#x_test = test_dataset.data.float()    # Convert images to float tensor
#y_test = test_dataset.targets          # Extract labels

train_subset = Subset(train_dataset_M, range(10000))
test_subset = Subset(test_dataset_M, range(1000))

batch_size = 10

train_loader_M = DataLoader(dataset=train_subset, batch_size=batch_size, shuffle=True)
test_loader_M = DataLoader(dataset=test_subset, batch_size=batch_size, shuffle=False)

#-----------------------------------------------------------------------------------------------------------------------------#

# Define transformations
transform_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
trainsubset = Subset(trainset, range(10000))
trainloader_C = torch.utils.data.DataLoader(trainsubset, batch_size=10, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)
testsubset = Subset(testset, range(1000))
testloader_C = torch.utils.data.DataLoader(testsubset, batch_size=10, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Extract features (images) and labels from the train set
#x_train_cifar = trainset.data.transpose((0, 3, 1, 2)).astype(np.float32) / 255.0  # Convert images to float tensor and normalize
#y_train_cifar = trainset.targets  # Extract labels

# Extract features (images) and labels from the test set
#x_test_cifar = testset.data.transpose((0, 3, 1, 2)).astype(np.float32) / 255.0  # Convert images to float tensor and normalize
#y_test_cifar = testset.targets  # Extract labels     

In [4]:
for images, labels in trainloader_C:
    print(images.shape)
    break
for images, labels in train_loader_M:
    print(images.shape)
    break

torch.Size([10, 3, 32, 32])
torch.Size([10, 1, 28, 28])


In [5]:
class Client1(nn.Module):
    def __init__(self, x_train, y_train):
        super(Client1, self).__init__()

        self.num_samples = x_train.shape[0]
        self.x_train = x_train
        self.labels = y_train
        self.batch = 10
        
        

        self.encoder_model = nn.Sequential(nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(1,1), padding='valid'),
                                    nn.ReLU(),
                                    nn.Conv2d(32, 64, kernel_size=(3,3), stride=(1,1), padding='valid'),
                                    nn.ReLU()
                                    )



    
        self.latent_model = nn.Sequential(
                                nn.MaxPool2d(kernel_size=(2,2)),
                                nn.Flatten()
                                 )

    
        self.decoder_model = nn.Sequential(nn.Linear(64*12*12, 10),
                                     nn.Softmax(dim=1))
        
        self.optimizer = optim.SGD(self.parameters(), lr=0.001)
        self.loss_function = nn.CrossEntropyLoss()

    def forward(self, inputs):
        outputs_encoder = self.encoder_model(inputs)
        outputs_latent = self.latent_model(outputs_encoder)
        latent_parameters = list(self.latent_model.parameters())
        outputs_class = self.decoder_model(outputs_latent)
        return outputs_class, latent_parameters

    
    def update(self, latent_parameters):
        # Get the parameters of the latent model
        latent_params = self.latent_model.parameters()
    
        # Iterate over the parameters and update them with new values
        for param, new_value in zip(latent_params, latent_parameters):
            param.data.copy_(new_value)
    
    
    def train_step(self, inputs, labels):
        
        #Forward pass
        logits, latent_parameters = self.forward(inputs)

        #Compute the loss
        loss = self.loss_function(logits, labels)

        #Zero the gradients
        self.optimizer.zero_grad()

        #Backward pass
        loss.backward()

        #updare the parameters
        self.optimizer.step()

        return loss.item(), latent_parameters

In [6]:
class Client2(nn.Module):
    def __init__(self, x_train, y_train):
        super(Client2, self).__init__()

        self.num_samples = x_train.shape[0]
        self.x_train = x_train
        self.labels = y_train
        self.batch = 10
        
        

        self.encoder_model = nn.Sequential(
                                    nn.Conv2d(3, 1, kernel_size=5, stride=1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(kernel_size=2, stride=2),
                                    nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(1,1), padding='valid'),
                                    nn.ReLU(),
                                    nn.Conv2d(32, 64, kernel_size=(3,3), stride=(1,1), padding='valid'),
                                    nn.ReLU()
                                    )



    
        self.latent_model = nn.Sequential(
                                nn.MaxPool2d(kernel_size=(2,2)),
                                nn.Flatten()
                                 )

    
        self.decoder_model = nn.Sequential(nn.Linear(1600, 10),
                                     nn.Softmax(dim=1))
        
        self.optimizer = optim.SGD(self.parameters(), lr=0.001)
        self.loss_function = nn.CrossEntropyLoss()

    def forward(self, inputs):
        outputs_encoder = self.encoder_model(inputs)
        outputs_latent = self.latent_model(outputs_encoder)
        latent_parameters = list(self.latent_model.parameters())
        outputs_class = self.decoder_model(outputs_latent)
        return outputs_class, latent_parameters


    def update(self, latent_parameters):
        # Get the parameters of the latent model
        latent_params = self.latent_model.parameters()
    
        # Iterate over the parameters and update them with new values
        for param, new_value in zip(latent_params, latent_parameters):
            param.data.copy_(new_value)
    
    def train_step(self, inputs, labels):
        
        #Forward pass
        logits, latent_parameters = self.forward(inputs)

        #Compute the loss
        loss = self.loss_function(logits, labels)

        #Zero the gradients
        self.optimizer.zero_grad()

        #Backward pass
        loss.backward()

        #updare the parameters
        self.optimizer.step()

        return loss.item(), latent_parameters

In [7]:
x_train = []
y_train = []
x_test = []
y_test = []

for images, labels in train_loader_M:
    x_train.append(images)
    y_train.append(labels)
    
for images_test, labels_test in test_loader_M:
    x_test.append(images_test)
    y_test.append(labels_test)

# Concatenate the batches to obtain the full datasets
x_train = torch.cat(x_train, dim=0)
y_train = torch.cat(y_train, dim=0)

x_test = torch.cat(x_test, dim=0)
y_test = torch.cat(y_test, dim=0)



In [8]:
x_train_C = []
y_train_C = []
x_test_C = []
y_test_C = []

for images, labels in trainloader_C:
    x_train_C.append(images)
    y_train_C.append(labels)
    
for images_test, labels_test in testloader_C:
    x_test_C.append(images_test)
    y_test_C.append(labels_test)

# Concatenate the batches to obtain the full datasets
x_train_C = torch.cat(x_train_C, dim=0)
y_train_C = torch.cat(y_train_C, dim=0)

x_test_C = torch.cat(x_test_C, dim=0)
y_test_C = torch.cat(y_test_C, dim=0)

In [10]:
#Instatiate your Client1 model
client1 = Client1(x_train, y_train)
client2 = Client2(x_train_C, y_train_C)

num_epochs = 10
latent_parameters = []

#training loop
for epoch in range(num_epochs):
    
    client1.train()
    client2.train()
    
    for i, ((images_M, labels_M), (images_C, labels_C)) in enumerate(zip(train_loader_M, trainloader_C)):
        loss_1, latent_parameters_1 = client1.train_step(images_M, labels_M)
        loss_2, latent_parameters_2 = client2.train_step(images_C, labels_C)
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss_1:.4f}")
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss_2:.4f}")
    if ((epoch+1)%2 == 0):
        print(epoch+1)
        latent_parameters =[(x + y) / 2 for x, y in zip(latent_parameters_1, latent_parameters_2)]
        client1.update(latent_parameters)
        client2.update(latent_parameters)
    
    latent_parameters = []

Epoch [1/10], Loss: 2.2739
Epoch [1/10], Loss: 2.3041
Epoch [2/10], Loss: 2.0908
Epoch [2/10], Loss: 2.3022
2
Epoch [3/10], Loss: 2.0885
Epoch [3/10], Loss: 2.3026
Epoch [4/10], Loss: 1.6269
Epoch [4/10], Loss: 2.3010
4
Epoch [5/10], Loss: 1.8855
Epoch [5/10], Loss: 2.3024
Epoch [6/10], Loss: 1.7509
Epoch [6/10], Loss: 2.3030
6
Epoch [7/10], Loss: 1.6108
Epoch [7/10], Loss: 2.3032
Epoch [8/10], Loss: 1.7979
Epoch [8/10], Loss: 2.3016
8
Epoch [9/10], Loss: 1.9284
Epoch [9/10], Loss: 2.3010
Epoch [10/10], Loss: 1.6208
Epoch [10/10], Loss: 2.3024
10


In [11]:
def test(inputs_1, labels_1, inputs_2, labels_2):
        logits_1, _ = client1.forward(inputs_1)
        logits_2, _ = client2.forward(inputs_2)
        

        _, predictions_1 = torch.max(logits_1, 1)
        _, predictions_2 = torch.max(logits_2, 1)
        
        acc_1 = torch.mean((predictions_1 == labels_1).float())
        acc_2 = torch.mean((predictions_2 == labels_2).float())

        return acc_1.item(), acc_2.item() 
print(test(x_test, y_test, x_test_C, y_test_C))

(0.6759999990463257, 0.13500000536441803)
